<h1><b>Statistique en Bioinformatique : </b> TME3 </h1>
<br>
L’objectif de ce TME est: 
<br>
<ul>
<li> objectif 1: comprendre les principaux etaphes de BLAST et PSIBLAST, </li>
<li> objectif 2: comprendre et comparer les algoritime de l'alignement progressive </li>
</ul>
<br>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><b>Soumission</b></p>
<ul>
<li>Renomer le fichier TME3.ipynb pour NomEtudiant1_NomEtudiant2.ipynb </li>
<li>Soumettre sur moodle </li>
</ul>
</div>


Nom etudiant 1 : GUILHON Henri
<br> 
Nom etudiant 2 : K.MURALI Sharane
<br>

<h3>Section 1: BLAST</h3>

<b> Exercice 1 </b>: À partir de l'application BLAST, disponible via le site de <a href="http://blast.ncbi.nlm.nih.gov/">NCBI</a>, cliquez sur protein Blast. Faites un BLAST de la séquence fasta/seq1.fasta. Expliquez le résultat graphique, la couleur de chaque hit et expliquez le tableau de résultat qu'est-ce que c'est chaque colonne?
<br>


<font color="red"> Réponse : </font>
<font color="blue"> On observe une liste de plusieurs séquences triées en fonction de leur taux de ressemblance par rapport à la séquence requête querry.
    
<font color="blue"> Une séquence de Plasmodium vivax donne un match à 100% soit un score maximal d'alignement.
   
<font color="blue"> La réprésentation graphique montre l'alignement de la séquence requête avec les 45 première séquences avec le meilleur score (qui est représenté par une couleur, ici toutes les séquences sont rouges, c'est à dire avec un score supérieur à 200).
    
<font color="blue"> Dans le tableau, on remarque plusieurs colonnes informatives:
    
<font color="blue"> Description : Nature (ADN,ARN,Protéine), Fonction et espèce de la séquence trouvée

<font color="blue"> Scientific Name : espèce dans laquelle on trouve la séquence
    
<font color="blue"> Max Score et Total Score : Score maximal que peut atteindre l'alignement entre la séquence query et la séquence alignée, avec le score d'alignement précis.
    
<font color="blue"> Query Cover : recouvrement de l'alignement sur la séquence requête : plus il est petit, moins il y a de nucléotides comparés pour le score
    
<font color="blue"> E-value (Expectetion-Value): nombre de matchs trouvés par chance. Indique la qualité des alignements, plus la e-value est basse, plus l'alignement est de bonne qualité. Zéro étant un alignement parfait.
    
<font color="blue"> Pourcentage d'identité : Score maximal / Score Total
    
<font color="blue"> Longueur d'alignement : Nombre de nulcéotides alignés entre les deux séquences.
    
<font color="blue"> Accession : lien de redirection vers la fiche de chaque séquence trouvée.
    </font>

<b>Exercice 2</b>: On s'intéresse à présent au gène FHIT chez l'homme (fragile histidine triad protein, identifiant P49789, télécharger du site  <a href="http://www.uniprot.org/"> Uniprot </a>). La protéine de ce gène est structurellement proche des galactose-1-phosphate uridylyltransferases. Faites un BLAST de ce gène contre la base SWISSPROT. Que constatez-vous?

<font color="red"> Réponse : </font>
<font color="blue">
On observe uniquement trois alignements avec environ 35% d'identité chacuns qu'on retrouve chez la souris, le rat et l'Homme
</font>

<b>Exercice 3</b>: Implémenter l'étape 0 du algorithme BLAST (indexer la base de données), utilise w=5 et les séquences du fichier database.fasta.

In [1]:
import numpy as np
import itertools as it
import os
import re
import pandas as pds

#variable initiation
database = 'fasta/database.fasta'
seqTestBlast = 'fasta/test.fasta'
input_blosum_f = 'BLOSUM62.txt'
aa = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'X']

#parameters
q = 21
gap = -5
w = 5


#For a simple test use:
databaseToy = 'fastaToy/databaseToy.fasta'
seqTestBlastToy = 'fastaToy/testToy.fasta'
wToy = 3

In [21]:
def etape0(fileName, w):
    """
    Database indexation
    Input1 fileName: fasta file name
    Input2 w:  word size
    Output1 index: dictionary containing indexed words
    Output2 dicoSeq: dictionary containing database entire sequences
    """
    index = {}
    dicoSeq = {}
    sequences = []
    
    with open(fileName, 'r') as f:
        seq  = ''
        for line in f:
            if line[0] == '>':
                sequences.append(seq)
                seq = ''
            else:
                seq += line.rstrip('\n')
        sequences.append(seq)
    #Le pop est présent pour retirer une séquence vide présente en première position
        sequences.pop(0)
    
    for i in range(len(sequences)):
        dicoSeq['s'+str(i+1)] = sequences[i].upper()
    for seq in dicoSeq:
        sub_sequences = [ dicoSeq[seq][i:i+w] for i in range(len(dicoSeq[seq])-w+1) ]
        for sub_seq in sub_sequences:
            index[sub_seq] = index.get(sub_seq,[]) + [ (seq, sub_sequences.index(sub_seq)) ]
        
    return index, dicoSeq

index_Toy, dicoSeq_Toy = etape0(databaseToy, wToy)

for i in index_Toy:
    print(i, index_Toy[i])
print(dicoSeq_Toy)
# {'AGH': [('s1', 0)], 'CGG': [('s3', 0)], 'GGH': [('s2', 1)]} 
# {'s1': 'AGHR', 's2': 'CGGHR', 's3': 'CGGH'}

AGH [('s1', 0)]
GHR [('s1', 1), ('s2', 2)]
CGG [('s2', 0), ('s3', 0)]
GGH [('s2', 1), ('s3', 1)]
GHA [('s3', 2)]
{'s1': 'AGHR', 's2': 'CGGHR', 's3': 'CGGHA'}


In [105]:
index, dicoSeq = etape0(database, w)

# Trop lourd à afficher
#for i in index:
#    print(i, index[i])

for i in dicoSeq:
    print(i, dicoSeq[i])

s1 KAIYNCSADHPDELTFSEGEVLVVDGEEDSDWWFGHVENESRRGVFPV
s2 KALYNCVADNADELTFSEGEMIIVDGEEDKEWWVGHIDGETRRGVFPV
s3 KALYNCVADNADELTFSEGEMIIVDGEEDKEWWVGHIDGETRRGVFPV
s4 KAVYNCVADNPDELTFAEGEVIVVDGEEDQEWWVAGPHRR
s5 KAVYNCVADNPDELTFAEGEVIVVDGEEDQEWWVAGPHRR
s6 KALYNCLADNPDELTFTEGEVIVVDGEEDQEWWKGHIEGDTRRGVFPT
s7 KALYNCVADNPDELTFSEGDVIIVDGEEDQEWWIGHIDGDGRKGAFPV
s8 KALYNCVADNPDELTFSEGDVIIVDGEEDQEWWIGHIDGDGRKGAFPV
s9 KALYNCVADNPDELTFSEGDVIIVDGEEDQEWWIGHIDGDGRKGAFPV
s10 KALYNCVADNPDELTLSEGDVIIVDGEEDQEWWIGHIDGESRKGAFPV
s11 KALYNCVADNPDELTFSEGDVIIVDGEEDQEWWIGHIDGESRKGAFPV
s12 KALYNCVADNPDELTFSEGDVIIVDGEEDQEWWIGHIDGESRKGAFPV
s13 KALYNCVADNPDELTFSEGDVIIVDGEEDQEWWIGHIDGESRKGAFPV
s14 KAIYHCSADNPDELTFSEGEIIVVDGEEDKEWWIGHIDGENRRGAFPV
s15 TIYDCQADNDDELTFMEGEVIVVTGEEDQEWWIGHIEGQERKGVFPV
s16 TIYDCQADNDDELTFVEGEVIIVTGEEDQEWWIGHIEGQERKGVFP
s17 TIYDCQADNDDELTFVEGEVIIVTGEEDQEWWIGHIEGQERKGVFP
s18 TIYDCQADNDDELTFIEGEVIIVTGEEDQEWWIGHIEGQERKGVFPV
s19 TIYDCQADNDDELTFIEGEVIIVTGEEDQEWWIGHIEGQERKGVFPV
s20 TIYDCQADNDDELTFIEGEVI

<b>Exercice 4</b>: Chercher la séquence query test.fasta dans la base indexé construite précédemment, entendez l'alignement pour avoir un alignement sans gap et de score > 12. Retournez les quatre meilleurs alignements.


In [3]:
#read Blosum

def readBlosum(nameFile):
    """
    Read substitution matrix file  
    Input1 fileName : blosum file name
    Output1 Blosum: a dictionary containing BLOSUM matrix, key is a pair of amino acids (alphabetic order) and value is the substitution value. 
    REMARK : You could eliminate the last four columns and last four rows of Blosum file --> NO we can't
    """
    Blosum = {}

    # Parcours du fichier 
    with open(nameFile, 'r') as in_f:
        i = 0
        # Parcours de chaque ligne
        for line in in_f:
            # Si on dépasse le nombre de lignes voulue on arrête 
            if i==q-1:
                break
            # Condition pour éviter de copier les premières lignes du fichier BLOSUM62.txt
            if line[0]=='#':
                continue
            if line[0]==' ':
                alphabet = line.split()[:q-1]
                print(alphabet)
                continue
            else:
                # Récupération de touts les caractères de la ligne sauf le premièr car c'est une lettre
                linebis = line.split()[1:q]
                
                # Affectation
                for j in range(len(linebis)):
                    Blosum[ alphabet[i]+alphabet[j] ] = int(linebis[j])
            i += 1
    
    return Blosum

blosum = readBlosum(input_blosum_f)
print(blosum["AA"])
#4
print(len(blosum))

['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
4
400


In [4]:
#Read the test.fasta file
def read_fasta (input_f):
    """
    Input1 input_f: fasta file name
    Output1 arr_seqs: List of sequences 
    REMARK : We keep only the sequences, identifiers are not necessary
    """
    arr_seqs = []
    
    
    #Parcours du fichier pour enregistrer les différentes séquences dans la liste
    with open(input_f, 'r') as in_f:
        seq  = ''
        for line in in_f:
            if line[0] == '>':
                arr_seqs.append(seq)
                seq = ''
            else:
                seq += line[:-1]
        arr_seqs.append(seq)

    #Le pop est présent pour retirer une séquence vide présente en première position
    arr_seqs.pop(0)
    
    return arr_seqs


Query = read_fasta(seqTestBlast)[0]
print(Query)

ALYDCEAENSDELTFNRGEIIQLIDRPDNDWWEGFIQSDSRRGMFP


In [52]:
def extendAln(pos1, pos2, sequence1, sequence2, matriceBlosum, scoremotif, w, ThrScore):
    """
    extend alignment left/right by keeping score above a threshold
    Input1 pos1 : position in the first sequence (Subject)
    Input2 pos2 : position in the second sequence (query)
    Input3 sequence1 : first sequence (Subject)
    Input4 sequence1 : second sequence (query)
    Input5 matriceBlosum: a dictionary containing BLOSUM matrix
    Input6 scoremotif: initial score 
    Input7 w: word size
    Input8 ThrScore: threshold (seuil)
    Output 1 finalScore: final score
    Output 2 algLength: alignment length
    Output 3 alnSeqs: Tuple containing aligned sequences
    """
    finalScore = scoremotif
    algLength  = w
    alnSeqs    = ()
    s1 = sequence1[pos1]
    s2 = sequence2[pos2]
    
    #Score = sum([ matriceBlosum[ nt+nt ] for nt in sequence2[pos2:pos2+w] ])
    extension = 1
    while finalScore > ThrScore:
            
            # Si l'extesion va au delà d'une des deux séquence on arrête la boucle
            if (pos1-extension<0 or pos2-extension<0 or pos1+extension>len(sequence1)-1 or pos2+extension>len(sequence2)-1): 
                break
            
            # Stockage des nucléotides des deux séquences à gauche(b) et à droite (a)
            s1be = sequence1[pos1-extension]
            s1ae = sequence1[pos1+extension]
            s2be = sequence2[pos2-extension]
            s2ae = sequence2[pos2+extension]
            
            # Alignement des extensions
            nt_al1 = s1be + s2be
            nt_al2 = s1ae + s2ae
            
            # Calcul des scores des extensions
            sb =  matriceBlosum[nt_al1]
            sa = matriceBlosum[nt_al2]
            
            # Ajout du score de gauche et extension de l'alignement à gauche
            finalScore += sb
            s1 = s1be + s1
            s2 = s2be + s2
            algLength += 1
            
            # Ajout du score de gauche et extension de l'alignement à gauche
            finalScore += sa
            s1 = s1 + s1ae
            s2 = s2 + s2ae
            algLength += 1

            # augmentation de l'extension pour aller un cran à côté
            extension += 1
    
    return finalScore, algLength, (s1,s2)

# On cherche à construire l'alignement entre une séquence requête et une séquence de la base de donnée
# En utilisant un algorithme d'extension de motif à partir d'un cluster, ie un alignement partiel de quelques
# Nucléotides / Acides Aminés entre les deux séquences
# Le but sera d'utiliser cette fonction dans blast lorsque l'on connaît ce premier alignement de motif entre les deux séquences
# Avec les indexes respectifs des positions dans ces séquences
res_Toy = extendAln(2, 2, dicoSeq_Toy['s2'], 'AGGHV' , blosum, 6, 1, 5)
print('Séquence dans la BDD:\t', dicoSeq['s2'] )
print('Séquence requête: \t AGGHV')
print (res_Toy)
#[20, 4, ('CGGH', 'AGGH')]

Séquence dans la BDD:	 KALYNCVADNADELTFSEGEMIIVDGEEDKEWWVGHIDGETRRGVFPV
Séquence requête: 	 AGGHV
(17, 5, ('CGGHR', 'AGGHV'))


In [54]:
def getWordsQuery(w, query, indexB):
    """
    Loking for indexed words in the query 
    Input1 w: word size
    Input2 query: sequence
    Input3 indexB: indexed database
    Output1 foundWords: Tuple containg found words, position in query, list of pairs (Subject id, Subject position)  
    """
    foundWords = []
    for i in indexB:
        if i in query:
            foundWords.append( (i, query.find(i), indexB[i]) )
        
    return foundWords


# Le but de cette fonction est de trouver les indexes de chaque motif de la base de donnée dans
# la séquence requête afin d'aligner ce motif avec celui d'une autre séquence dans la BDD pour
# établir un cluster pour la fonction extendAlin
foundWords_Toy = getWordsQuery(wToy, 'AGGHV', index_Toy)

print(foundWords_Toy)
#[('GGH', 1, [('s2', 1)])]

for word in foundWords_Toy[0][2]:
    print(word[0], dicoSeq_Toy[word[0]])

[('GGH', 1, [('s2', 1), ('s3', 1)])]
s2 CGGHR
s3 CGGHA


In [59]:
foundWords = getWordsQuery(w, Query, index)
for i in foundWords:
    print(i)

('DELTF', 10, [('s1', 11), ('s2', 11), ('s3', 11), ('s4', 11), ('s5', 11), ('s6', 11), ('s7', 11), ('s8', 11), ('s9', 11), ('s11', 11), ('s12', 11), ('s13', 11), ('s14', 11), ('s15', 10), ('s16', 10), ('s17', 10), ('s18', 10), ('s19', 10), ('s20', 10), ('s21', 10), ('s22', 10), ('s23', 10), ('s24', 10), ('s25', 10), ('s26', 10), ('s27', 11), ('s31', 5), ('s56', 11), ('s58', 11), ('s150', 11), ('s151', 11), ('s159', 11), ('s171', 11), ('s258', 11), ('s630', 11), ('s631', 11), ('s632', 11), ('s1453', 11), ('s2375', 11), ('s2397', 11), ('s3005', 6), ('s3845', 11), ('s4073', 10), ('s4186', 10), ('s4192', 11), ('s4193', 11), ('s4194', 11), ('s4195', 11), ('s4196', 11), ('s4197', 11), ('s4198', 11), ('s4199', 11), ('s4200', 11), ('s4201', 11), ('s4202', 11), ('s4203', 11), ('s4204', 11), ('s4205', 11), ('s4206', 11), ('s4207', 11), ('s4217', 11), ('s4230', 11), ('s4246', 11), ('s4248', 11), ('s4421', 11), ('s4422', 11), ('s4531', 11), ('s4723', 6), ('s4876', 8), ('s4877', 8), ('s4878', 10), 

In [90]:
#Implement BLAST algorithm

def Blast(foundWords, indexedDB, Query, dicoSeq, T, w, matriceBlosum):
    """
    input1 foundWords: Tuple containg found words, position in query, list of pairs (Subject id, Subject position
    input2 indexedDB: indexed database
    input3 query: query sequence
    input4 dicoSeq: dictionary containing database sequences
    input5 T: Threahold score
    input6 w: word size
    Input7 matriceBlosum: a dictionary containing BLOSUM matrix
    output1 alignedSeq: Tuple of aligned sequences
    """
    Scores_alignements    = []
    Longueurs_alignements = []
    alignedSeq            = []
    database_seq_keys = []
    
    # Pour chaque motif de taille w stocké dans la base de données que l'on retrouve dans la séquence requête
    for word in foundWords:
        
        # On stocke chaque valeur séparément
        motif            = word[0]
        seq_index        = word[1]
        database_indexes = word[2]
        
        
        # On aligne les séquences de la base de données avec la séquence requête en partant des indices stockés
        for DBseq_name, DBseq_index in database_indexes:
            scoremotif = sum([ matriceBlosum[ nt+nt ] for nt in Query[seq_index:seq_index+w] ])
            
            
            score, align_lenght, alignements = extendAln(seq_index, DBseq_index, Query, dicoSeq[DBseq_name], matriceBlosum, scoremotif, w, T)
            
            Scores_alignements.append(score)
            Longueurs_alignements.append(align_lenght)
            alignedSeq.append(alignements)
            database_seq_keys.append(DBseq_name)
    
    # Transformation en array pour utiliser la bibliothèque numpy
    Scores_alignements = np.array(Scores_alignements)
    Longueurs_alignements = np.array(Longueurs_alignements)
    alignedSeq = np.array(alignedSeq)
    database_seq_keys = np.array(database_seq_keys)
    
    # Les 4 alignements avec le score le plus élévé
    nb = min(4,len(Scores_alignements))
    arguments = np.argpartition(Scores_alignements, -nb)[-nb:]
    # print(arguments)
    
    # Construction des 
    Key_Access  = database_seq_keys[arguments]
    Alignements = alignedSeq[arguments]
    Query_Cover = np.round(100 * Longueurs_alignements[ arguments ] / np.array([len(dicoSeq[key]) for key in Key_Access ]),3)
    Max_Scores  = [sum([matriceBlosum[nt+nt] for nt in Query_part[0] ]) for Query_part in Alignements]
    Scores      = Scores_alignements[ arguments ]
    Identité    = [ 100*sum([ align[0][i]==align[1][i] for i in range(len(align[0])) ])/len(align[0]) for align in Alignements ]
    
    return Key_Access, Alignements, Query_Cover, Max_Scores, Scores, np.round(Identité,3)

# Pour la séquence QueryToy avec la base de données Toy:
key_access ,alignedSeq, Query_Cover, max_scores, scores, identité = Blast(foundWords_Toy, index_Toy, 'AGGHV', dicoSeq_Toy, 12, wToy, blosum )
print('Les meilleurs alignements de séquences retournés par BLAST: \nPour la requête Toy sur la databaseToy')

data_frame = []
for i in range(len(alignedSeq)):
    data_frame.append( [alignedSeq[i][0], key_access[i], Query_Cover[i], max_scores[i], scores[i], identité[i]] )

colonnes = ['Séquence de la Base de données alignée avec la séquence Query', 'Clé d\'accession', 'Query_Cover', 'Score_Max', 'Score', '% d\'Identité']
df = pds.DataFrame( np.array(data_frame), columns=colonnes)
df

Les meilleurs alignements de séquences retournés par BLAST: 
Pour la requête Toy sur la databaseToy


,Séquence de la Base de données alignée avec la séquence Query,Clé d'accession,Query_Cover,Score_Max,Score,% d'Identité
0,AGG,s2,100.0,16,26,66.667
1,AGG,s3,100.0,16,26,66.667


In [91]:
key_access ,alignedSeq, Query_Cover, max_scores, scores, identité = Blast(foundWords, index, Query, dicoSeq, 12, w, blosum )
print('Les 4 meilleurs alignements de séquences retournés par BLAST: \nPour la Query test sur databaseTest')

data_frame = []
for i in range(len(alignedSeq)):
    data_frame.append( [alignedSeq[i][0], key_access[i],  Query_Cover[i], max_scores[i], scores[i], identité[i]] )

colonnes = ['Séquence de la Base de données alignée avec la séquence Query', 'Clé d\'accession', 'Query_Cover', 'Score_Max', 'Score', '% d\'Identité']
indexes = ['Séquence '+str(i+1) for i in range(4)]
df = pds.DataFrame( np.array(data_frame), columns=colonnes, index=indexes)
df

Les 4 meilleurs alignements de séquences retournés par BLAST: 
Pour la Query test sur databaseTest


,Séquence de la Base de données alignée avec la séquence Query,Clé d'accession,Query_Cover,Score_Max,Score,% d'Identité
Séquence 1,ALYDCEAENSDELTFNRGEIIQLIDRPDNDWWEGFIQSD,s4880,97.727,217,114,46.154
Séquence 2,ALYDCEAENSDELTFNRGEIIQLIDRPDNDWWEGFIQ,s4880,93.182,207,115,48.649
Séquence 3,SDELTFNRGEIIQLIDRPDNDWWEGFIQSDSRRGMFP,s2338,85.417,205,125,37.838
Séquence 4,SDELTFNRGEIIQLIDRPDNDWWEGFIQSDSRRGMFP,s2339,85.417,205,125,37.838


<h3>Section 2: PSI-BLAST</h3>
On considère à nouveau le gène FHIT chez l'homme. 
<BR>
<b> Exercice 1 </b>:
Faites un PSI-BLAST de ce gène contre la base SWISS-PROT. Utilisez un seuil pour l’évalue de 1e-6.
Que constatez-vous lors de la 1ère itération concernant les protéines de la famille des galactose-1-phosphate uridylyltransferases?


<font color="red"> Réponse : </font>
<font color="blue">
On retrouve les 3 similarités de séquence avec la souris, le rat, l'Homme mais aussi avec une protéine de C.elegans avec un pourcentage d'indentité de 27% à la première itération de PSI-BLAST.
    
</font>

<b> Exercice 2 </b>:
Que constatez-vous lors de la 2eme itération? Lors de la 5eme itération?


<font color="red"> Réponse : </font>
<font color="blue">
A la seconde itération, aucune séquence n'est ajoutée au résultat, à la 5eme aussi
</font>

<b> Exercice 3 </b>:
À partir de quelle itération peut-on considérer qu'il y a eu une convergence de l'algorithme PSI- BLAST? Pour quoi ?

<font color="red"> Réponse : </font>
<font color="blue">
Dès la seconde, car on trouve des résultats avec la matrice PSSM construite avec les séquences trouvées, si celles-ci restentn identiques, alors on trouvera toujours les mêmes séquences quelque soit le nombre de répétitions.
</font>

<h3>Section 3: Alignement Multiple</h3>


<b> Exercice 1 </b>:
Accédez à l'application <a href="http://www.ebi.ac.uk/Tools/msa/clustalo/">Clustal omega</a> et copiez les 7 séquences du fichier 7sequences.fasta. Expliquez, dans quel ordre les séquences ont été alignées.

<font color="red"> Réponse : </font>
<font color="blue">
Les séquences ont été alignées de haut en bas en fonction de leur distance phylogénétique relatives à leurs taux d'identité.
</font>

<b> Exercice 2 </b>:
Faite une fonction pour calculer le score d’un alignement multiple. Le score de chaque position est la somme des tous les scores obtenus par BLOSUM62, et le score final est la somme des scores de chaque position divisée par la taille du alignement.

In [ ]:
#Read multi fasta
def extraireAllFastaMulti(fileName):  
    """
    Read a FASTA file with several sequences
    input1 fileName: file name
    output1 IdSeq: list of sequences IDs
    output2 Seqs: sequences    
    """ 
    arr_seqs = []
    names = []
    
    #Parcours du fichier pour enregistrer les différentes séquences dans la liste
    with open(fileName, 'r') as in_f:
        seq  = ''
        name = ''
        for line in in_f:
            if line[0] == '>':
                names.append(name)
                arr_seqs.append(seq)
                seq = ''
            else:
                seq += line[:-1]
        names.append(name)
        arr_seqs.append(seq)

    #Le pop est présent pour retirer une séquence vide présente en première position
    arr_seqs.pop(0)
    names.pop(0)
    return arr_seqs, names

In [94]:
def SP_score(SeqsM, blosum, gap):
    """
    Compute MSA score based on a substitution matrix and gap penality
    input1 SeqsM: list of aligned sequences
    input2 blosum: substitution matrix
    input3 gap: gap penality
    output1 score: MSA score
    """
    score = 0
    
    # indice de colonne
    for column_i in range(len( SeqsM[0])):
        
        # extraction de la colonne
        column = [st[column_i] for st in SeqsM ]
        #print(column)
        
        # toutes les combinaisons
        for tool in it.combinations(column, 2):
            a, b = tool
            #print(tool)
            
            # changement de nomenclature des gaps
            if a == '-' or b == '-':
                    score += gap
            # ajout du score de chaque paire pour une colonne donnée
            else:
                score += blosum[ a + b ]
            
    return score / len(SeqsM[0])

SeqsM = ['ACG', 'A-G', 'ACD']
score = SP_score(SeqsM, blosum, gap)
print (score)
#5.0

5.0


<b> Exercice 3 </b>: Aligner les séquences du fichiers algnMult2.fasta avec  <a href="http://www.ebi.ac.uk/Tools/msa/clustalo/">Clustal omega</a>. Utiliser la fonction développé precedement  pour calculer le score du alignement. Quelle est la séquence la plus divergente? 

In [95]:
# Dans le cas où l'on on gardé le format clustal_o, on parse les informations dedans : chaque séquence alignée
def read_clustal_O(file_name):
    align = {}
    with open( file_name, 'r' ) as f:
        for line in f.readlines()[1:]:
            if line[0] not in [' ', '\t', '\n']:
                splited_line = line.split()
                align[ splited_line[0] ] = align.get(splited_line[0], '') + splited_line[1]
    return align

# Si le fichier au format clustal est dans le répertoire courant, on l'ouvre et l'on parse les séquences alignées
# on utilise des boucles try pour éviter que les erreurs arrêtent le code
try:
    # on récupère tous les noms de fichiers au format clustal dans le répertoire commun
    clustal_alignements_files = [string for string in os.listdir() if re.search( "clustal_num$", string)!=None]
    
    # On prend le premier
    aligned_seq = read_clustal_O(clustal_alignements_files[0])
    
    # On calcule le score d'alignement des séquences
    score = SP_score(list(aligned_seq.values()), blosum, gap)
    print('Score global d\'alignement:', score)
except:
    pass

# Score global d'alignement: 202.08680675163623

Score global d'alignement: 202.08680675163623


In [101]:
# Dans le cas où l'on a demandé l'alignement en fasta, ce qui est plus simple :
try:
    seq, names = extraireAllFastaMulti("aln-fasta.txt")

    score = SP_score(seq, blosum, gap)
    print('Score global d\'alignement:', score)
except:
    pass

<font color="red"> Réponse : </font>
<font color="blue">
Il est difficile de répondre à cette question car l'on ne peut identifier précisément une séquence plus ou moins divergente par rapport aux autres dans un alignement à partir d'un score d'alignement.
</font>

<b> Exercice 4 </b>:
Comparer les trois outils  <a href="http://www.ebi.ac.uk/Tools/msa/clustalo/">Clustal omega</a>,  <a href="http://mafft.cbrc.jp/alignment/server/">MAFT</a>  et <a href="http://www.ebi.ac.uk/Tools/msa/muscle/">Muscle</a> en utilisant les séquences du fichiers algnMult.fasta. Utiliser la fonction développée précédemment pour calculer le score de chaque alignement. Puis répéter la comparaison avec les séquences du fichier algnMult2.fasta. Quel outil a eu la meilleure performance?

</font>
<font color="blue"> L'algorithme Muscle donne ici de meilleurs résultats d'alignements par rapport à Clustal_O et MAFT
    
<font color="blue"> Par ailleurs MAFT donne un résultat d'alignement très bas
</font>

In [100]:
# On parse ici les fichiers d'alignement issus des trois outils Clustal_Omega, MAFT et Muscle
# On effectue dans des boucles try pour éviter que les erreurs arrêtent le code
# Les fichiers de résultats d'alignements seront aussi fournis

try:
    clustal_alignements_files = [string for string in os.listdir() if re.search( "clustal_num$", string)!=None]
    aligned_seq = read_clustal_O(clustal_alignements_files[1])
    score = SP_score(list(aligned_seq.values()), blosum, gap)
    print('Score global d\'alignement Clustal-O:\t', score)
except:
    pass
    
try:
    clustal_alignements_files = [string for string in os.listdir() if re.search( "aln$", string)!=None]
    aligned_seq = read_clustal_O(clustal_alignements_files[0])
    score = SP_score(list(aligned_seq.values()), blosum, gap)
    print('Score global d\'alignement MAFT :\t', score)
except:
    pass

try:
    clustal_alignements_files = [string for string in os.listdir() if re.search( "clw$", string)!=None]
    aligned_seq = read_clustal_O(clustal_alignements_files[0])
    score = SP_score(list(aligned_seq.values()), blosum, gap)
    print('Score global d\'alignement Muscle :\t', score)
except:
    pass

# Score global d'alignement Clustal-O:	 202.08680675163623
# Score global d'alignement MAFT :	     62.69350380096752
# Score global d'alignement Muscle :	 206.85837950138503

Score global d'alignement Clustal-O:	 202.08680675163623
Score global d'alignement MAFT :	 62.69350380096752
Score global d'alignement Muscle :	 206.85837950138503
